In [1]:
import numpy as np
import pandas as pd
# import csv
# import matplotlib.cm as cm
# import matplotlib.mlab as mlab
# import matplotlib.pyplot as plt
# import scipy as sp
import scipy.io as sio
# import scipy.stats as stats
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import Imputer
from math import log2
from math import sqrt

In [2]:
SPAM = 'dist/spam_data.mat'
TRAIN_CENSUS = 'hw5_census_dist/train_data.csv'
TRAIN_TITANIC = 'hw5_titanic_dist/titanic_training.csv'
TEST_TITANIC = 'hw5_titanic_dist/titanic_testing_data.csv' 
TEST_CENSUS = 'hw5_census_dist/test_data.csv'

In [3]:
spam_data = sio.loadmat(SPAM)
census_tdata = pd.read_csv(TRAIN_CENSUS)
titanic_tdata = pd.read_csv(TRAIN_TITANIC)
census_test = pd.read_csv(TEST_CENSUS)
titanic_test = pd.read_csv(TEST_TITANIC)

In [4]:
"""TITANIC TRAINING DATA"""
"""Removed ticket and cabin features"""
titanic_tdata_filtered = titanic_tdata.drop('ticket', 1)
titanic_tdata_filtered = titanic_tdata_filtered.drop('cabin', 1)

"""Row 707 is empty in titanic_training.csv"""
titanic_tdata_filtered = titanic_tdata_filtered.dropna(how='all')

"""Map categories to binary variables [1(a)]"""
tv = DictVectorizer()
vectorized_titanic = tv.fit_transform(titanic_tdata_filtered.to_dict(orient='records'))

# """Replaced unknown values with mean of features [2(b)]"""
# timp = Imputer()
# imputed_titanic = timp.fit_transform(vectorized_titanic)

# ttdata = pd.DataFrame(data=imputed_titanic.toarray(), columns=tv.get_feature_names(), dtype='object')

ttdata = pd.DataFrame(data=vectorized_titanic.toarray(), columns=tv.get_feature_names(), dtype='object')

ttdata['pclass'].fillna(ttdata['pclass'].mode(), inplace=True)
ttdata['sex=male'].fillna(ttdata['sex=male'].mode(), inplace=True)
ttdata['sex=female'].fillna(ttdata['sex=female'].mode(), inplace=True)
ttdata['embarked'].fillna(ttdata['embarked'].mode(), inplace=True)


ttdata['age'].fillna(int(ttdata['age'].mean()), inplace=True)
ttdata['sibsp'].fillna(int(ttdata['sibsp'].mean()), inplace=True)
ttdata['parch'].fillna(int(ttdata['parch'].mean()), inplace=True)
ttdata['fare'].fillna(int(ttdata['fare'].mean()), inplace=True)


In [5]:
"""TITANIC TEST DATA"""

"""Removed ticket and cabin features"""
titanic_test_filtered = titanic_test.drop('ticket', 1)
titanic_test_filtered = titanic_test_filtered.drop('cabin', 1)

"""Map categories to binary variables [1(a)]"""
tv = DictVectorizer()
vectorized_ttest = tv.fit_transform(titanic_test_filtered.to_dict(orient='records'))

"""Replaced unknown values with mean of features [2(b)]"""
timp = Imputer()
timputed = timp.fit_transform(vectorized_ttest)

ttestdata = pd.DataFrame(data=timputed.toarray(), columns=tv.get_feature_names(), dtype='object')

In [6]:
"""CENSUS TRAINING DATA"""

"""Map categories to binary variables [1(a)]"""
cv = DictVectorizer()
vectorized_census = cv.fit_transform(census_tdata.to_dict(orient='records'))

"""Replaced unknown values with mean of features [2(b)]"""
cimp = Imputer()
imputed_census = cimp.fit_transform(vectorized_census)

ctdata = pd.DataFrame(data=imputed_census.toarray(), columns=cv.get_feature_names(), dtype='object')


"""CENSUS TEST DATA"""

"""Map categories to binary variables [1(a)]"""
cv = DictVectorizer()
vectorized_ctest = cv.fit_transform(census_test.to_dict(orient='records'))

"""Replaced unknown values with mean of features [2(b)]"""
cimp = Imputer()
imputed_census_test = cimp.fit_transform(vectorized_ctest)

ctestdata = pd.DataFrame(data=imputed_census_test.toarray(), columns=cv.get_feature_names(), dtype='object')

In [7]:
npstdata = spam_data['training_data']
npstlabels = spam_data['training_labels']

stcombo = np.append(npstdata, npstlabels.T, axis=1)
features = [str(i) for i in range(npstdata.shape[1])]
features.append('label')
stdata = pd.DataFrame(data=stcombo, columns=features, dtype='object')

stestdata = pd.DataFrame(data=spam_data['test_data'], columns=features[:-1], dtype='object')

# Decision Tree Abstraction
## Class InternalNode
* ### State
    * Node left, right
    * split_feature [split rule]
    * split_value [split rule]
* ### Methods
    * `predict(data)`
        * given a data point, chooses left or right child based on the split rule
        * traverses starting from this node
    * `is_leaf() { return False }`

## Class LeafNode
* ### State
    * label
* ### Methods
    * `is_leaf() { return True }`

## Class DecisionTree
* ### State
    * root
* ### Methods
    * `impurity(left_label_hist, right_label_hist)`
        * calculates the entropy of a split
    * `segmenter(data, labels)`
        * finds the best split rule using impurity()
        * many different types of segmenters
    * `train(train_data, train_labels, depth_limited)`
        * grows the decision tree
        * uses segmenter to find the best splits
    * `predict(data)`
        * given a data point, traverses the tree starting at the root
    

In [28]:
class LeafNode:
    def __init__(self, label):
        self.label = label

    def predict(self, data):
        return self.label

    def is_leaf(self):
        return True

    def __repr__(self):
        return 'Leaf({})'.format(self.label)

class InternalNode:
    def __init__(self, left, right, split_feature, split_value):
        self.left, self.right = left, right
        self.split_feature = split_feature
        self.split_value = split_value

    def predict(self, data):
        if data[self.split_feature] < self.split_value:
            return self.left.predict(data)
        return self.right.predict(data)

    def is_leaf(self):
        return False

    def __repr__(self):
        return 'InternalNode({}, {})'.format(self.split_feature, self.split_value)

class DecisionTree:
    def __init__(self, root=None):
        self.root = root

    def logsp(self, x, y):
        if x == 0:
            return 0
        return x * log2(x / (x + y))

    def impurity(self, C, D, nC, nD):
        c, d = nC - C, nD - D
        return -(self.logsp(C, D) + self.logsp(D, C) + self.logsp(c, d) + self.logsp(d, c)) / (nC + nD)

    def count(self, data, label):
        nC, nD = 0, 0
        for lbl in data[label]:
            if lbl == 0:
                nC += 1
            else:
                nD += 1
        return nC, nD

    def segmenter(self, data, label, is_random_forest=False, m=0):
        """
        For splits on a feature f with a value v,
            left will have samples with f values strictly less than v
            right will have samples with f values greater than or equal to v
        """
        if is_random_forest:
            if m <= 0:
                m = int(sqrt(len(data.axes[1]) - 1))
            elif m < 1:
                m = int((len(data.axes[1]) - 1) * m)
            features = data.drop(label, 1).sample(m, axis=1)
        else:
            features = data.drop(label, 1).axes[1]

        min_entropy, splitf, splitv, spliti = float('inf'), None, None, 0

        for f in features:
            sorted_data_by_f = data.sort_values(f)

            iterv = sorted_data_by_f[f].__iter__()
            iterl = sorted_data_by_f[label].__iter__()

            nC, nD = self.count(sorted_data_by_f, label)

            # keeps track of which value-label pair we're on
            v, lbl, i = next(iterv), next(iterl), 0

            # keeps track of class counts on LEFT
            C, D = 0, 0

            # don't check if all data lies on one side, since then this node should be a leaf (base case of train)
            beta_iter = sorted_data_by_f.drop_duplicates(subset=f, keep='first')[f].__iter__()
            next(beta_iter)

            for beta in beta_iter:
                try:
                    while v != beta:
                        if lbl == 0:
                            C += 1
                        else:
                            D += 1
                        i += 1
                        v, lbl = next(iterv), next(iterl)
                except StopIteration:
                    continue
                entropy = self.impurity(C, D, nC, nD)
                if entropy < min_entropy:
                    min_entropy, splitf, splitv, spliti = entropy, f, beta, i
            # no need to check split on max(beta)+1 since then all elements are on one split (node should be leaf)

        if splitf == None:
            return None, None, splitf, splitv #, spliti

        sorted_data = data.sort_values(splitf)
        left = sorted_data.iloc[:spliti]
        right = sorted_data.iloc[spliti:]

        return left, right, splitf, splitv #, spliti

    def train(self, train_data, label, depth_limited=float('inf'), is_random_forest=False, m=0):
        def grow_tree(train_data, label, depth_limited=float('inf'), is_random_forest=False, m=0):
            labels = train_data[label].unique()
            if depth_limited == 0 or len(labels) == 1:
                return LeafNode(labels[0])

            left_data, right_data, split_feature, split_value = self.segmenter(train_data, label, is_random_forest, m)

            if left_data is None:
                lbl = train_data[label].value_counts().argmax()
                return LeafNode(lbl)

            left = grow_tree(left_data, label, depth_limited - 1, is_random_forest, m)
            right = grow_tree(right_data, label, depth_limited - 1, is_random_forest, m)
            return InternalNode(left, right, split_feature, split_value)
        self.root = grow_tree(train_data, label, depth_limited, is_random_forest, m)

    def predict(self, data):
        return self.root.predict(data)

class RandomForest:
    def __init__(self, n=150):
        self.n = n
        self.forest = []

    def train(self, train_data, label, depth_limited=float('inf'), m=0, bagging=0):
        if bagging == 0:
             bagging = len(train_data)
        for i in range(self.n):
            t = DecisionTree()
            t.train(train_data.sample(bagging, replace=True), label, depth_limited, True, m)
            self.forest.append(t)
            if i % (self.n // 10)  == 0:
                print(i)

    def predict(self, data):
        predictions, count0, count1 = [], 0, 0
        for t in self.forest:
            predictions.append(t.predict(data))
            if predictions[-1] == 0:
                count0 += 1
            else:
                count1 += 1
        if count0 > count1:
            return 0
        return 1

In [ ]:
shuffled_ttdata = ttdata.sample(frac=1)
tvnum = int(len(shuffled_ttdata) // 10)
vsttdata = shuffled_ttdata[:tvnum]
tsttdata = shuffled_ttdata[tvnum:]

tlabel = 'survived'
rf = RandomForest(150)
rf.train(tsttdata, tlabel, depth_limited=10)

correct = 0
for _, vsample in vsttdata.iterrows():
    if rf.predict(vsample) == vsample[tlabel]:
        correct += 1
print(correct / len(vsttdata), correct, len(vsttdata))

0
15
30


In [34]:
shuffled_ttdata = ttdata.sample(frac=1)
tvnum = int(len(shuffled_ttdata) // 10)
vsttdata = shuffled_ttdata[:tvnum]
tsttdata = shuffled_ttdata[tvnum:]

tlabel = 'survived'
tdt = DecisionTree()
tdt.train(tsttdata, tlabel)

correct = 0
for _, vsample in vsttdata.iterrows():
    if tdt.predict(vsample) == vsample[tlabel]:
        correct += 1
print(correct / len(vsttdata), correct, len(vsttdata))

0.8383838383838383 83 99


In [232]:
shuffled_ttdata = ttdata.sample(frac=1)
tvnum = int(len(shuffled_ttdata) // 10)
vsttdata = shuffled_ttdata[:tvnum]
tsttdata = shuffled_ttdata[tvnum:]

tlabel = 'survived'
tdt = DecisionTree()
tdt.train(tsttdata, tlabel)

correct = 0
for _, vsample in vsttdata.iterrows():
    if tdt.predict(vsample) == vsample[tlabel]:
        correct += 1
print(correct / len(vsttdata), correct, len(vsttdata))

0.7575757575757576 75 99


In [146]:
kaggle_tdt = DecisionTree()
kaggle_tdt.train(ttdata, 'survived')

In [61]:
f = open('titanic.csv', 'w')
f.write("Id,Category\n")
i = 1
for _, vsample in ttestdata.iterrows():
    f.write("{},{}\n".format(i, int(kaggle_tdt.predict(vsample))))
    i += 1
f.close()

In [242]:
shuffled_ctdata = ctdata.sample(frac=1)
cvnum = int(len(shuffled_ctdata) // 10)
vsctdata = shuffled_ctdata[:cvnum]
tsctdata = shuffled_ctdata[cvnum:]

clabel = 'label'
cdt = DecisionTree()
cdt.train(tsctdata, clabel)

correct = 0
for _, vsample in vsctdata.iterrows():
    if cdt.predict(vsample) == vsample[clabel]:
        correct += 1
print(correct / len(vsctdata), correct, len(vsctdata))

0.80440097799511 2632 3272


In [58]:
kaggle_cdt = DecisionTree()
kaggle_cdt.train(ctdata, 'label')

In [60]:
f = open('census.csv', 'w')
f.write("Id,Category\n")
i = 1
for _, vsample in ctestdata.iterrows():
    f.write("{},{}\n".format(i, int(kaggle_cdt.predict(vsample))))
    i += 1
f.close()

In [120]:
shuffled_stdata = stdata.sample(frac=1)
svnum = int(len(shuffled_stdata) // 10)
vsstdata = shuffled_stdata[:svnum]
tsstdata = shuffled_stdata[svnum:]

slabel = 'label'
sdt = DecisionTree()
sdt.train(tsstdata, slabel)

correct = 0
for _, vsample in vsstdata.iterrows():
    if sdt.predict(vsample) == vsample[clabel]:
        correct += 1
print(correct / len(vsstdata), correct, len(vsstdata))

0.7624472573839662 1807 2370


In [123]:
kaggle_sdt = DecisionTree()
kaggle_sdt.train(stdata, 'label')

In [125]:
f = open('spam.csv', 'w')
f.write("Id,Category\n")
i = 0
for _, vsample in stestdata.iterrows():
    f.write("{},{}\n".format(i, int(kaggle_sdt.predict(vsample))))
    i += 1
f.close()

In [59]:
correct = 0
clabel = 'label'
for _, vsample in ctdata.iterrows():
    if kaggle_cdt.predict(vsample) == vsample[clabel]:
        correct += 1
print(correct / len(ctdata), correct, len(ctdata))

1.0 32724 32724


In [174]:
small_titanic = ttdata[:30]
small_tdt = DecisionTree()
# small_titanic['survived'].unique()
small_tdt.train(small_titanic, 'survived')

In [177]:
p = small_tdt.predict(ttdata.iloc[345])
print(p, ttdata['survived'][345])

1.0 1.0


In [166]:
# q = []
# q.append(small_tdt.root)
# while len(q) > 0:
#     n = q.pop(0)
#     if not n.is_leaf():
#         print(n.split_feature, n.split_value, '\n')
#         q.append(n.left)
#         q.append(n.right)
#     else:
#         print('label: ', n.label, '\n')

In [143]:
k, a, b = 'survived', 'apple', 'banana'
d = [{k: 0, a: 0, b: 1},
     {k: 0, a: 1, b: 77},
     {k: 0, a: 1, b: 4},
     {k: 1, a: 1, b: 1}, 
     {k: 1, a: 1, b: 2}]

# d = [{k: 0, a: 0, b: 1},
#      {k: 1, a: 0, b: 1},
#      {k: 1, a: 0, b: 1}]

dfd = pd.DataFrame(d)
tdt = DecisionTree()
tdt.train(dfd, k)
q = []
q.append(tdt.root)
while len(q) != 0:
    n = q.pop(0)
    print(n)
    if not n.is_leaf():
        q.append(n.left)
        q.append(n.right)

InternalNode(banana, 4)
InternalNode(apple, 1)
Leaf(0)
Leaf(0)
Leaf(1)


In [205]:
raw_data = {'first_name': ['Jason', 'Jason', 'Tina', 'Jake', 'Amy', 'Zach'],
        'last_name': ['Miller', 'Miller', 'Ali', 'Milner', 'Cooze', 'Miller'],
        'age': [42, 42, 36, 24, 73, 4],
        'preTestScore': [4, 4, 31, 2, 3, 100],
        'postTestScore': [25, 25, 57, 62, 70, 100]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
print(df)
print(df.drop_duplicates(subset='first_name', keep='first'))

  first_name last_name  age  preTestScore  postTestScore
0      Jason    Miller   42             4             25
1      Jason    Miller   42             4             25
2       Tina       Ali   36            31             57
3       Jake    Milner   24             2             62
4        Amy     Cooze   73             3             70
5       Zach    Miller    4           100            100
  first_name last_name  age  preTestScore  postTestScore
0      Jason    Miller   42             4             25
2       Tina       Ali   36            31             57
3       Jake    Milner   24             2             62
4        Amy     Cooze   73             3             70
5       Zach    Miller    4           100            100
